In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import GraphHelpers as GH
import ETFunctions as et
import MathsUtilities as MUte

## Bring in the data needed

In [2]:
MetData = pd.read_pickle('BroadFieldsMet.pkl')
DaylightIndex = MetData.IncomingRadnMJ.where(MetData.IncomingRadnMJ>0.002)
DaylightIndex.dropna(inplace=True)
DaylightIndex = DaylightIndex.index.tolist()
MetData.loc[:,'satVP'] = [et.saturated_vapor_pressure(MetData.loc[x,'AirTemperature'])/10 for x in MetData.index]
MetData.loc[:,'VP'] = [MetData.loc[x,'satVP']*MetData.loc[x,'RelativeHumidity']/100 for x in MetData.index]
MetData.loc[:,'VPD'] = MetData.satVP.subtract(MetData.VP)

DailyMet = MetData.IncomingRadnMJ.resample('D').sum()
DailyMet = pd.DataFrame(index=DailyMet.index,data=DailyMet.values,columns=['Radn'])
DailyMet.loc[:,'Temp'] = MetData.AirTemperature.resample('D').mean()
DailyMet.loc[:,'RH'] = MetData.RelativeHumidity.resample('D').mean()
DailyMet.loc[:,'PET'] = [et.Priestly_TaylorEO(DailyMet.loc[x,'Radn'],
                                    DailyMet.loc[x,'Temp'],
                                    1.3,'Total') for x in DailyMet.index]
DailyMet.loc[:,'VPD'] = MetData.loc[DaylightIndex,'VPD'].resample('D').mean()
MinVPD = 0.1
DailyMet.loc[:,'VPD'] = DailyMet.loc[:,'VPD'].where(DailyMet.loc[:,'VPD']>MinVPD,MinVPD)
AccumPET = DailyMet.PET.cumsum()

In [3]:
#Bring in soil water use data.
LucWU = pd.read_pickle('.\LucerneAndPasture\DataFrames\LucWU.pkl').cumsum()
RyeWU = pd.read_pickle('.\LucerneAndPasture\DataFrames\RyeWU.pkl').cumsum()
LucWU = LucWU.where(LucWU.notna(),0)
RyeWU = RyeWU.where(RyeWU.notna(),0)

In [4]:
StartDate = LucWU.index.min()
EndDate = LucWU.index.max()
AllDays = AccumPET.loc[StartDate:EndDate].index
LucWUDaily = pd.DataFrame(index = AllDays,columns = LucWU.columns)
RyeWUDaily = pd.DataFrame(index = AllDays,columns = RyeWU.columns)
LucWU.loc[:,'AccumPET'] = [AccumPET[x] for x in LucWU.index]
RyeWU.loc[:,'AccumPET'] = [AccumPET[x] for x in RyeWU.index]
for col in LucWUDaily.columns:
    LucWUDaily.loc[:,col] = np.interp(AccumPET.loc[AllDays].values,
                                      LucWU.AccumPET,
                                      pd.to_numeric(LucWU.loc[:,col]))
for col in RyeWUDaily.columns:
    RyeWUDaily.loc[:,col] = np.interp(AccumPET.loc[AllDays].values,
                                      RyeWU.AccumPET,
                                      pd.to_numeric(RyeWU.loc[:,col]))

In [5]:
LucWU = -LucWUDaily.diff()
RyeWU = -RyeWUDaily.diff()

WheatDaylightIndex = np.load('.\Wheat\DataFrames\DaylightIndex.npy')
WheatHourlyWU = pd.read_pickle('.\Wheat\DataFrames\HourlyWUmm.pkl')
WheatWU = pd.concat([WheatHourlyWU.reindex(WheatDaylightIndex).resample('d').sum()],axis=1,keys=['Wheat'],names=['Crop'])

FodderBeetDaylightIndex = np.load('.\FodderBeet\DataFrames\DaylightIndex.npy')
FodderBeetHourlyWU = pd.read_pickle('.\FodderBeet\DataFrames\HourlyWUmm.pkl')
FodderBeetWU = pd.concat([FodderBeetHourlyWU.reindex(FodderBeetDaylightIndex).resample('d').sum()],axis=1,keys=['FodderBeet'],names=['Crop'])

OatsDaylightIndex = np.load('.\Oats\DataFrames\DaylightIndex.npy')
OatsHourlyWU = pd.read_pickle('.\Oats\DataFrames\HourlyWUmm.pkl')
OatsWU = pd.concat([OatsHourlyWU.reindex(OatsDaylightIndex).resample('d').sum()],axis=1,keys=['Oats'],names=['Crop'])
#OatsWU = pd.concat([pd.read_pickle('.\Oats\DataFrames\DailyWU.pkl')],axis=1,keys=['Oats'],names=['Crop'])

BarleyDaylightIndex = np.load('.\Barley\DataFrames\DaylightIndex.npy')
BarleyHourlyWU = pd.read_pickle('.\Barley\DataFrames\HourlyWUmm.pkl') 
BarleyWU = pd.concat([BarleyHourlyWU.reindex(BarleyDaylightIndex).resample('d').sum()],axis=1,keys=['Barley'],names=['Crop'])
#BarleyWU.columns = BarleyWU.columns.swaplevel('Irrigation','Cultivar')

WU = pd.concat([WheatWU,FodderBeetWU,LucWU,RyeWU,OatsWU,BarleyWU],axis=1)
WU.dropna(how='all',inplace=True)
GH.SortFrame(WU)

In [6]:
RTData = pd.read_csv('C:\\GitHubRepos\\ApsimX\\Tests\\Validation\\SoilWater\\a_block_obs_soilwater.txt',
                     delim_whitespace=True,skiprows=[1])
RTData.set_index(['SimulationName','Clock.Today'],inplace=True)

In [7]:
ABlock = pd.read_csv('C:\\GitHubRepos\\CropWaterAnalysis\\Evaporation\\AblockResiduesSWC.dat',
            skiprows=[0,2,3])
ABlock.set_index(['TIMESTAMP'],inplace=True)
ABlock.drop(['RECORD'],inplace=True,axis=1)
ABlock.columns.name = 'ColumnID' 
ABlockIndex = pd.read_excel('C:\\GitHubRepos\\CropWaterAnalysis\\Evaporation\\DataIndex.xlsx')
ABlockIndex.set_index('ColumnID',inplace=True)
JoinIndex = ABlockIndex.transpose().loc[('Treatment','Depth','Block'),:]

In [8]:
Intrum = pd.concat([JoinIndex,ABlock])
Intrum = Intrum.transpose()
Intrum.set_index(['Treatment','Depth','Block'],inplace=True,drop=True)
ResidueExptSWC = Intrum.transpose().iloc[:,:56].copy()
GH.SortFrame(ResidueExptSWC)

In [9]:
Treatments = ResidueExptSWC.columns.get_level_values(0).drop_duplicates().values
Depths = ResidueExptSWC.columns.get_level_values(1).drop_duplicates().values

In [13]:
Treatments

array(['mown and baled', 'mown stubble lying',
       'mown, baled and cultivated', 'stubble standing'], dtype=object)

In [ ]:
Graph = plt.figure(figsize=(18,20))
pos=1
for D in Depths:
    Graph.add_subplot(1,3,pos)
    for T in Treatments:
        plt.plot(ResidueExptSWC.index,ResidueExptSWC.loc[:,(T,D)])
    pos+=1

In [ ]:
Evap = pd.DataFrame(index = RTData.index,
                    data = RTData.loc[:,'Soil.SoilWater.SWmm(1)'].diff().values,
                    columns = ['0-20'])
Evap.loc[:,'0-40'] = RTData.loc[:,'Soil.SoilWater.SWmm(1)'].diff() + RTData.loc[:,'Soil.SoilWater.SWmm(2)'].diff() 
Evap.loc[:,'0-60'] = RTData.loc[:,'Soil.SoilWater.SWmm(1)'].diff() + RTData.loc[:,'Soil.SoilWater.SWmm(2)'].diff() + RTData.loc[:,'Soil.SoilWater.SWmm(3)'].diff() 

In [ ]:
DailyRain = MetData.Rainfall.resample('D').sum()
Rainfall = [DailyRain[x[1]]<0.5 for x in Evap.index]

In [ ]:
Treatments = Evap.index.get_level_values(0).drop_duplicates().values
for treat in Treatments:
    plt.plot(Evap.loc[treat,'0-20'])
#Evap.loc[Rainfall,:].plot()

In [ ]:
Evap.loc[treat,'0-20']